In [1]:
import cv2
from inference.inference import *
def video_inference(video_file):
    cap = cv2.VideoCapture(video_file)
    model = setting_model()
    if cap.isOpened():
        while True:
            ret, img = cap.read()
            result_img = inference(img, model)
            cv2.imshow(video_file, result_img)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
    else:
        print("can't open video.")
    cap.release()
    cv2.destroyAllWindows()

In [2]:
import os
PATH = os.getcwd()
if __name__ =="__main__":
    video = './media/test_video/London_test.mp4'
    video_inference(video)

# Import the required modules
import cv2
from IPython.display import clear_output
import time
import PIL.Image
from cStringIO import StringIO
import IPython.display
import numpy as np

gain: 0.7494145199063232
pad: (0.0, 140.14051522248243)
gain: 0.7494145199063232
pad: (0.0, 140.14051522248243)
gain: 0.7494145199063232
pad: (0.0, 140.14051522248243)
gain: 0.7494145199063232
pad: (0.0, 140.14051522248243)
gain: 0.7494145199063232
pad: (0.0, 140.14051522248243)
gain: 0.7494145199063232
pad: (0.0, 140.14051522248243)
gain: 0.7494145199063232
pad: (0.0, 140.14051522248243)
gain: 0.7494145199063232
pad: (0.0, 140.14051522248243)
gain: 0.7494145199063232
pad: (0.0, 140.14051522248243)
gain: 0.7494145199063232
pad: (0.0, 140.14051522248243)
gain: 0.7494145199063232
pad: (0.0, 140.14051522248243)
gain: 0.7494145199063232
pad: (0.0, 140.14051522248243)
gain: 0.7494145199063232
pad: (0.0, 140.14051522248243)
gain: 0.7494145199063232
pad: (0.0, 140.14051522248243)
gain: 0.7494145199063232
pad: (0.0, 140.14051522248243)
gain: 0.7494145199063232
pad: (0.0, 140.14051522248243)
gain: 0.7494145199063232
pad: (0.0, 140.14051522248243)
gain: 0.7494145199063232
pad: (0.0, 140.14051522

gain: 0.7494145199063232
pad: (0.0, 140.14051522248243)
gain: 0.7494145199063232
pad: (0.0, 140.14051522248243)
gain: 0.7494145199063232
pad: (0.0, 140.14051522248243)
gain: 0.7494145199063232
pad: (0.0, 140.14051522248243)
gain: 0.7494145199063232
pad: (0.0, 140.14051522248243)
gain: 0.7494145199063232
pad: (0.0, 140.14051522248243)
gain: 0.7494145199063232
pad: (0.0, 140.14051522248243)
gain: 0.7494145199063232
pad: (0.0, 140.14051522248243)
gain: 0.7494145199063232
pad: (0.0, 140.14051522248243)
gain: 0.7494145199063232
pad: (0.0, 140.14051522248243)
gain: 0.7494145199063232
pad: (0.0, 140.14051522248243)
gain: 0.7494145199063232
pad: (0.0, 140.14051522248243)
gain: 0.7494145199063232
pad: (0.0, 140.14051522248243)
gain: 0.7494145199063232
pad: (0.0, 140.14051522248243)
gain: 0.7494145199063232
pad: (0.0, 140.14051522248243)
gain: 0.7494145199063232
pad: (0.0, 140.14051522248243)
gain: 0.7494145199063232
pad: (0.0, 140.14051522248243)
gain: 0.7494145199063232
pad: (0.0, 140.14051522

gain: 0.7494145199063232
pad: (0.0, 140.14051522248243)
gain: 0.7494145199063232
pad: (0.0, 140.14051522248243)
gain: 0.7494145199063232
pad: (0.0, 140.14051522248243)
gain: 0.7494145199063232
pad: (0.0, 140.14051522248243)
gain: 0.7494145199063232
pad: (0.0, 140.14051522248243)
gain: 0.7494145199063232
pad: (0.0, 140.14051522248243)
gain: 0.7494145199063232
pad: (0.0, 140.14051522248243)
gain: 0.7494145199063232
pad: (0.0, 140.14051522248243)
gain: 0.7494145199063232
pad: (0.0, 140.14051522248243)
gain: 0.7494145199063232
pad: (0.0, 140.14051522248243)
gain: 0.7494145199063232
pad: (0.0, 140.14051522248243)
gain: 0.7494145199063232
pad: (0.0, 140.14051522248243)
gain: 0.7494145199063232
pad: (0.0, 140.14051522248243)
gain: 0.7494145199063232
pad: (0.0, 140.14051522248243)
gain: 0.7494145199063232
pad: (0.0, 140.14051522248243)
gain: 0.7494145199063232
pad: (0.0, 140.14051522248243)
gain: 0.7494145199063232
pad: (0.0, 140.14051522248243)
gain: 0.7494145199063232
pad: (0.0, 140.14051522

In [1]:
# Import the required modules
import cv2
from IPython.display import clear_output
import time
import PIL.Image
from io import BytesIO
import IPython.display
import numpy as np
#Use 'jpeg' instead of 'png' (~5 times faster)
def showarray(a, fmt='jpeg'):
    
    f = BytesIO()
    PIL.Image.fromarray(a).save(f, fmt)
    IPython.display.display(IPython.display.Image(data=f.getvalue()))

In [2]:
def get_frame(cam):
    # Capture frame-by-frame
    ret, frame = cam.read()
    
    #flip image for natural viewing
    frame = cv2.flip(frame, 1)
    
    return frame



In [3]:
cam = cv2.VideoCapture(0)
try:
    while(True):
        t1 = time.time()
        
        # Capture frame-by-frame
        frame = get_frame(cam)

        # Convert the image from OpenCV BGR format to matplotlib RGB format
        # to display the image
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        showarray(frame)
        t2 = time.time()

        print("%f FPS" % (1/(t2-t1)))
        # Display the frame until new frame is available
        clear_output(wait=True)
except KeyboardInterrupt:
    cam.release()
    print("Stream stopped")


Stream stopped
